In [37]:
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
tqdm.pandas()

c:\python3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Read Data and Add Index

In [39]:
ccag = ['QHP017 ADVANTAGE HEALTH NYC', 'DISCOUNT CARD AMERICAN HEALTH COPR', 'AMWINS MEDICARE MAPH QHP', 'AARONS INC']
company = ['VANTAGE HEALTH PLAN', 'AMERICAN HEALTH COPR', 'AMWINS RX', 'AARONS CONCRETE']

In [40]:
df = pd.DataFrame({
    'ccag':ccag,
    'company':company
})

In [41]:
df['index'] = df.index
df.insert(0, 'index',  df.pop('index'))

In [42]:
df

,index,ccag,company
0,0,QHP017 ADVANTAGE HEALTH NYC,VANTAGE HEALTH PLAN
1,1,DISCOUNT CARD AMERICAN HEALTH COPR,AMERICAN HEALTH COPR
2,2,AMWINS MEDICARE MAPH QHP,AMWINS RX
3,3,AARONS INC,AARONS CONCRETE


# TF-IDF Algorithm

In [43]:
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [44]:
vectorizer = TfidfVectorizer(min_df = 0,token_pattern='(?u)\\b\\w+\\b')
train_matrix = vectorizer.fit_transform(ccag+company)
query_matrix = vectorizer.transform(company)

In [45]:
ccag_matrix = train_matrix[0:len(ccag)].toarray()
company_matrix = query_matrix.toarray()

In [46]:
def compute_similarity(row, ccag_matrix, company_matrix):
    index = row['index']
    return round(np.dot(ccag_matrix[index], company_matrix[index].T) * 100)

In [47]:
df['%similarity'] = df.progress_apply(compute_similarity, args=(ccag_matrix,company_matrix), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 4006.02it/s]


# Final Output

In [48]:
df

,index,ccag,company,%similarity
0,0,QHP017 ADVANTAGE HEALTH NYC,VANTAGE HEALTH PLAN,14
1,1,DISCOUNT CARD AMERICAN HEALTH COPR,AMERICAN HEALTH COPR,69
2,2,AMWINS MEDICARE MAPH QHP,AMWINS RX,28
3,3,AARONS INC,AARONS CONCRETE,41


# Fuzz

In [38]:
t0 = 'AARONS INC'
t1 = 'AARONS CONCREATE'
print("sort", fuzz.token_sort_ratio(t0, t1))
print("set", fuzz.token_set_ratio(t0, t1))

sort 69
set 75
